Goal here is to build a basic model, reiterate later upon study of other solutions, revisit with different valuation method

**Rough draft to be edited for clarity**

In NFL, what is the best way to distribute value across cap?

Clustering (assumptions): minimum / rookie contract / free agent / franchise tag / outliers

53+ roster spots.

How many of each contract can be afforded, and what is the optimal distribution?

Then account for positional value?


cluster by salary, Measure AV/salary for all and determine optimal distribution of salaries by contract type by AV (as a basic measure of player value)

**Import the data**

need to use webscraping. 

Use pfr and spotrac as the sources. in future I think I would use these files to make a sql database

Not sure which value to use for spotrac, average only appears available for active (2022-) contracts, historical values seem 
subject to be skewed by  things like contract restructures -- sorting by 'cash' seems to provide desired values, though this may be misleading for players whose salary is being paid by former team -- might make sense to go player by player and use yearly totals from 'career earnings', which appears to be cash values paid across all teams for a season

spotrac and pfr seem to be consistent with which players were on which rosters historically, though spotrac adds has players across categories like injury IR, dead cap, practice squad etc. and classifies based on which category a player is in at the end of the season.

In [1]:
from bs4 import BeautifulSoup
import requests

In [4]:
html_page = requests.get('https://www.pro-football-reference.com/teams/buf/2021_roster.htm') # Make a get request to retrieve the page
soup = BeautifulSoup(html_page.content, 'html.parser') # Pass the page contents to beautiful soup for parsing

In [5]:
soup.prettify

<bound method Tag.prettify of 
<!DOCTYPE html>

<html class="no-js" data-root="/home/pfr/build" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202207051" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://quantcast.mgr.consensu.org'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, '/choice.js')
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    v

In [13]:
warning = soup.find('div', class_="placeholder")
warning

<div class="placeholder"></div>

In [34]:
stats_table = warning.nextSibling.nextSibling
stats_table

'\n\n<div class="table_container" id="div_roster">\n    \n    <table class="per_match_toggle sortable stats_table" id="roster" data-cols-to-freeze=",2">\n    <caption>Roster Table</caption>\n    \n\n   <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup>\n   <thead>      \n      <tr>\n         <th aria-label="No." data-stat="uniform_number" scope="col" class=" poptip sort_default_asc center" data-tip="Uniform number" >No.</th>\n         <th aria-label="Player" data-stat="player" scope="col" class=" poptip sort_default_asc show_partial_when_sorting left" >Player</th>\n         <th aria-label="Age" data-stat="age" scope="col" class=" poptip sort_default_asc center" data-tip="Player\'s age on December 31st of that year" >Age</th>\n         <th aria-label="Pos" data-stat="pos" scope="col" class=" poptip sort_default_asc left" data-tip="<b>Position</b><br>In player and team season stats,<br>Capitals indicates primary starter.<br>Lower-case means part-time s

In [37]:
parsed_table = soup.find_all('table')
parsed_table

[]

In [21]:
type(stats_table)

bs4.element.Comment

**Inspect dataset, prepare data appropriately**

Want to have dataframe that at least has player position({'str':float64}), salary(real()), value(av()), and yearly salary cap (real()) formatted properly

**Cluster salaries**

I think this is mostly done using an ai?

**Evaluate all players value compared to salary**

value_per_dollar = av / salary

**Visualize and evaluate**
analyze by clusters(look up how to cluster, pretty sure this is done with basic ai) (use mean. assumpt: In NFL, team control to block player movement prevents teams from paying above market rate, and that teams would do so to extract value. However, they can't so much of how it is distributed is (assumed)random. so each team is as likely to acquire players anywhere within that cluster of contracts.

Use salary earned rather than cap figure to account for fact that that number fluctuates to accomodate teams to their cap needs, but doesn't reflect cost as effectively as salary)

perc_cap = salary / salary_cap 

order clusters by value, how many of each can exist on a team, best distribution given need for at least 53(how many could become better understood)

**Ideas to explore**

better ways to evaluate value than av

how many players are needed on a roster, does that affect distribution?

evaluate by positional value (did it have underlying effect in previous inquiry?)

find teams historically that fit similar construction based on first inquiry, and compare their relative performance (AV, W/L, super bowl, etc)/
find all possible clusters, look for their occurences in nfl teams and compare by their relative performance (AV, W/L, super bowl, etc)

**Sources**

pfr scraping - https://stmorse.github.io/journal/pfr-scrape-python.html

spotrac scraping - https://github.com/the-data-analyst/python_portfolio_web_scraper-spotrac/blob/master/webScraping_sample.py

scrapy - https://scrapy.org/